In [1]:
import pandas as pd
import pickle 
from scipy.stats import poisson

In [3]:
dict_table = pickle.load(open('dict_table', 'rb'))
df_historial_data = pd.read_csv('clean_fifa_world_historica.csv')
df_fixture = pd.read_csv('clean_fifa_world2022_fixture.csv')

In [4]:
years = [1930, 1934, 1938, 1950, 1954, 1958, 1962,1966, 1970, 1974, 1978,1982,
         1986, 1990, 1994, 1998, 2002,2006, 2010, 2014, 2018]

for year in years:
    print(year, len(df_historial_data[df_historial_data['Year'] == year]))

1930 18
1934 17
1938 18
1950 22
1954 26
1958 35
1962 32
1966 32
1970 32
1974 38
1978 38
1982 52
1986 52
1990 52
1994 52
1998 64
2002 64
2006 64
2010 64
2014 64
2018 64


### Calcular team Strengh

In [5]:
dict_table.keys()

dict_keys(['Group A', 'Group B', 'Group C', 'Group D', 'Group E', 'Group F', 'Group G', 'Group H'])

In [6]:
df_historial_data


,HomeTeam,VisitTeam,Year,HomeGoals,VisitGoals,TotalGoals
0,France,Mexico,1930,4,1,5
1,Argentina,France,1930,1,0,1
2,Chile,Mexico,1930,3,0,3
3,Chile,France,1930,1,0,1
4,Argentina,Mexico,1930,6,3,9
...,...,...,...,...,...,...
895,Russia,Croatia,2018,2,2,4
896,France,Belgium,2018,1,0,1
897,Croatia,England,2018,2,1,3
898,Belgium,England,2018,2,0,2


In [11]:
# Fuerza de un equipo, para saber cuantos goles anota
# creo un dataframe para vel los goles 
df_home = df_historial_data[['HomeTeam', 'HomeGoals', 'VisitGoals']]
df_away = df_historial_data[['VisitTeam', 'HomeGoals', 'VisitGoals']]
df_away
df_home

,HomeTeam,HomeGoals,VisitGoals
0,France,4,1
1,Argentina,1,0
2,Chile,3,0
3,Chile,1,0
4,Argentina,6,3
...,...,...,...
895,Russia,2,2
896,France,1,0
897,Croatia,2,1
898,Belgium,2,0


In [9]:
df_away.loc[df_away['VisitTeam'] == 'El Salvador']

,VisitTeam,HomeGoals,VisitGoals
201,El Salvador,3,0
203,El Salvador,4,0
204,El Salvador,2,0
321,El Salvador,10,1
323,El Salvador,1,0
325,El Salvador,2,0


In [12]:
df_home.loc[df_home['HomeTeam'] == 'El Salvador']

,HomeTeam,HomeGoals,VisitGoals


In [13]:
# De esta manera separo los equipos "locales" con su respectivos goles anotados y concedidos
# De la misma manera los equipos "visitantes" con sus respectivos goles recibidos y anotados 
df_home = df_home.rename(columns={'HomeTeam':'Team', 'HomeGoals': 'GoalsScored', 'VisitGoals': 'GoalsConceded'})
df_away = df_away.rename(columns={'AwayTeam':'Team', 'HomeGoals': 'GoalsConceded', 'VisitGoals': 'GoalsScored'})

In [14]:
df_home.groupby('Team').sum()

,GoalsScored,GoalsConceded
Team,,
Algeria,5,10
Angola,0,1
Argentina,111,50
Australia,7,13
Austria,32,22
...,...,...
Uruguay,68,36
Wales,2,1
West Germany,95,34


In [15]:
# Concatenamos los dataframes gome y away, agrego un gropup by para agrupar por equipo 
# mean() se encarga de calcuular los promedios de goles 
df_team_strength = pd.concat([df_home, df_away]).groupby(['Team']).mean()
df_team_strength

C:\Users\Luis\AppData\Local\Temp\ipykernel_11700\3164584905.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_team_strength = pd.concat([df_home, df_away]).groupby(['Team']).mean()


,GoalsScored,GoalsConceded
Team,,
Algeria,0.833333,1.666667
Angola,0.000000,1.000000
Argentina,1.913793,0.862069
Australia,1.000000,1.857143
Austria,2.285714,1.571429
...,...,...
Uruguay,2.125000,1.125000
Wales,2.000000,1.000000
West Germany,2.500000,0.894737


In [16]:
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,0.833333,1.666667
Angola,0.000000,1.000000
Argentina,1.913793,0.862069
Australia,1.000000,1.857143
Austria,2.285714,1.571429
...,...,...
Uruguay,2.125000,1.125000
Wales,2.000000,1.000000
West Germany,2.500000,0.894737


In [17]:
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,0.833333,1.666667
Angola,0.000000,1.000000
Argentina,1.913793,0.862069
Australia,1.000000,1.857143
Austria,2.285714,1.571429
...,...,...
Uruguay,2.125000,1.125000
Wales,2.000000,1.000000
West Germany,2.500000,0.894737


In [64]:
df_team_strength.to_csv('promedio.csv' )

### Function predict_points

In [18]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        # goals_scored * goals_conceded
        lamb_home = df_team_strength.at[home,'GoalsScored'] * df_team_strength.at[away,'GoalsConceded']
        lamb_away = df_team_strength.at[away,'GoalsScored'] * df_team_strength.at[home,'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        for x in range(0,11): #number of goals home team
            for y in range(0, 11): #number of goals away team
                p = poisson.pmf(x, lamb_home) * poisson.pmf(y, lamb_away)
                if x == y:
                    prob_draw += p
                elif x > y:
                    prob_home += p
                else:
                    prob_away += p
        
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return (points_home, points_away)
    else:
        return (0, 0)

### Tester Funtion


In [19]:
predict_points('Argentina', 'China')

(2.9503615583149947, 0.02176208144567273)

In [20]:
predict_points('China', 'Brazil')

(0.012397705021160634, 2.958407514925697)

### Predicting World Cup

In [21]:
df_fixture

,HomeTeam,score,VisitTeam,Year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
...,...,...,...,...
59,Winners Match 51,Match 59,Winners Match 52,2022
60,Winners Match 57,Match 61,Winners Match 58,2022
61,Winners Match 59,Match 62,Winners Match 60,2022
62,Losers Match 61,Match 63,Losers Match 62,2022


In [22]:
# Divido el ficture por cada fase para sacar los partidos
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semi = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

In [106]:
df_fixture

,HomeTeam,score,VisitTeam,Year
0,Qatar,Match 1,Ecuador,2022
1,Senegal,Match 2,Netherlands,2022
2,Qatar,Match 18,Senegal,2022
3,Netherlands,Match 19,Ecuador,2022
4,Ecuador,Match 35,Senegal,2022
...,...,...,...,...
59,Winners Match 51,Match 59,Winners Match 52,2022
60,Winners Match 57,Match 61,Winners Match 58,2022
61,Winners Match 59,Match 62,Winners Match 60,2022
62,Losers Match 61,Match 63,Losers Match 62,2022


In [23]:
for group in dict_table:
    print(group)
    
for group in dict_table:
    print(dict_table[group]['Team'].values)

Group A
Group B
Group C
Group D
Group E
Group F
Group G
Group H
['Qatar (H)' 'Ecuador' 'Senegal' 'Netherlands']
['England' 'Iran' 'United States' 'Wales']
['Argentina' 'Saudi Arabia' 'Mexico' 'Poland']
['France' 'Australia' 'Denmark' 'Tunisia']
['Spain' 'Costa Rica' 'Germany' 'Japan']
['Belgium' 'Canada' 'Morocco' 'Croatia']
['Brazil' 'Serbia' 'Switzerland' 'Cameroon']
['Portugal' 'Ghana' 'Uruguay' 'South Korea']


In [24]:
for group in dict_table:
    # obtengo la lista de los equipos de cada grupo
    teams_in_group = dict_table[group]['Team'].values
    # se obtiene los partidos los equipos del mismo grupo
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['HomeTeam'].isin(teams_in_group)]
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['HomeTeam'], row['VisitTeam']
        # ejecuto la funcion que calcula la fuerza de cada equipo
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

In [26]:
# mostrar tabla actualizada
dict_table['Group G']

,Team,Pts
0,Brazil,8.0
1,Switzerland,6.0
2,Cameroon,3.0
3,Serbia,1.0


In [27]:
# Octavos de final 
df_fixture_knockout

,HomeTeam,score,VisitTeam,Year
48,Winners Group A,Match 49,Runners-up Group B,2022
49,Winners Group C,Match 50,Runners-up Group D,2022
50,Winners Group D,Match 52,Runners-up Group C,2022
51,Winners Group B,Match 51,Runners-up Group A,2022
52,Winners Group E,Match 53,Runners-up Group F,2022
53,Winners Group G,Match 54,Runners-up Group H,2022
54,Winners Group F,Match 55,Runners-up Group E,2022
55,Winners Group H,Match 56,Runners-up Group G,2022


In [29]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    runners_up = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'Winners {group}':group_winner,
                                 f'Runners-up {group}':runners_up}, inplace=True)

df_fixture_knockout['winner'] = '?'
df_fixture_knockout

,HomeTeam,score,VisitTeam,Year,winner
48,Netherlands,Match 49,Wales,2022,?
49,Argentina,Match 50,Denmark,2022,?
50,France,Match 52,Mexico,2022,?
51,England,Match 51,Ecuador,2022,?
52,Germany,Match 53,Croatia,2022,?
53,Brazil,Match 54,Uruguay,2022,?
54,Belgium,Match 55,Spain,2022,?
55,Portugal,Match 56,Switzerland,2022,?


In [30]:
# Definimos el canador de cada llave 
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['HomeTeam'], row['VisitTeam']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

In [31]:
# Usamos la funcion para establecer al ganador
get_winner(df_fixture_knockout)

,HomeTeam,score,VisitTeam,Year,winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Mexico,2022,France
51,England,Match 51,Ecuador,2022,England
52,Germany,Match 53,Croatia,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Belgium,Match 55,Spain,2022,Belgium
55,Portugal,Match 56,Switzerland,2022,Portugal


In [32]:
# Funcion 4tos de ffina

def update_table(df_fixture_round_1, df_fixture_round_2):
    for index, row in df_fixture_round_1.iterrows():
        winner = df_fixture_round_1.loc[index, 'winner']
        match = df_fixture_round_1.loc[index, 'score']
        df_fixture_round_2.replace({f'Winners {match}':winner}, inplace=True)
    df_fixture_round_2['winner'] = '?'
    return df_fixture_round_2

In [33]:
update_table(df_fixture_knockout, df_fixture_quarter)

,HomeTeam,score,VisitTeam,Year,winner
56,Germany,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Belgium,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


In [34]:
# definimos los ganadores de los 4tos
get_winner(df_fixture_quarter)

,HomeTeam,score,VisitTeam,Year,winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Argentina
58,Belgium,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,England


In [35]:
update_table(df_fixture_quarter, df_fixture_semi)

,HomeTeam,score,VisitTeam,Year,winner
60,Argentina,Match 61,Brazil,2022,?
61,England,Match 62,Portugal,2022,?


In [36]:
get_winner(df_fixture_semi)

,HomeTeam,score,VisitTeam,Year,winner
60,Argentina,Match 61,Brazil,2022,Brazil
61,England,Match 62,Portugal,2022,England


In [38]:

update_table(df_fixture_semi, df_fixture_final)

,HomeTeam,score,VisitTeam,Year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,England,2022,?


In [39]:

get_winner(df_fixture_final)

,HomeTeam,score,VisitTeam,Year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,England,2022,England
